<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Ablations/FDAContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re

In [ ]:
fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/FDAPartialContext.csv")

In [ ]:
fda_df

In [ ]:
drugs_list = []
drugs_trade_list = []
drugs_yes_no_responses = []
yes_no_prompts = []
answer_short_list = []
groundtruth_list = []
set_list = []

for index, row in tqdm(fda_df.iterrows(), total=fda_df.shape[0]):


    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Cardiotoxicity?'
        Now, answer with just one word: Yes or No"""

    #confidence_prompt = """Are you confident in your answer? Err on the side of saying 'No' if you are not sure.
    #    Now, answer with just one word: Yes or No"""

    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content":row["Reference"]},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    print(yes_no_response.choices[0].message.content.strip()," ",row["GT_Label"])
    drugs_list.append(row["genrname"])
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    yes_no_prompts.append(yes_no_prompt)
    set_list.append(row["set_id"])

    drugs_trade_list.append(row["Drug_Name"])
    answer_short_list.append(row["Answer_short"])
    groundtruth_list.append(row["GT_Label"])
    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"trade_name":drugs_trade_list,"binary_determination":drugs_yes_no_responses,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"short_answer":answer_short_list,"ground_truth":groundtruth_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Monday_FDA_Context_temp"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"trade_name":drugs_trade_list,"binary_determination":drugs_yes_no_responses,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"short_answer":answer_short_list,"ground_truth":groundtruth_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Monday_FDA_Context_final.csv")

In [ ]:
final_df

In [ ]:
final_df["answer_binary"] = final_df["ground_truth"].map({"no":0,"most":1})

In [ ]:
final_df["pred_binary"] = final_df["binary_determination"].map({"No":0,"Yes":1})
final_df["their_pred_binary"] = final_df["short_answer"].map({"No":0,"Yes":1})

In [ ]:
result_df = final_df.groupby('generic_name').agg({
    'answer_binary': 'max',
    'pred_binary': 'max',
    'their_pred_binary': 'max'
}).reset_index()

In [ ]:
result_df

In [ ]:
result_df

In [ ]:
df_to_use = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/uniTox Ratings/FinaluniTox/Friday_DICT_direct_4o_cardiotoxicity_comprehensive.csv")

In [ ]:
df_to_use["set_id"]=df_to_use["urls"].str.replace("https://fdalabel.fda.gov:8443/fdalabel/services/spl/set-ids/","").str.replace("https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/","").str.replace("/spl-doc","")

In [ ]:
result_df = result_df.merge(df_to_use, on="generic_name", how="inner")

In [ ]:
result_df["our_pred_binary"] = result_df["binary_determination"].map({"No":0,"Yes":1})

In [ ]:
result_df

In [ ]:
print((result_df["our_pred_binary"]==result_df["answer_binary"]).sum()/len(result_df))

In [ ]:
print((result_df["pred_binary"]==result_df["answer_binary"]).sum()/len(result_df))

In [ ]:
print((result_df["their_pred_binary"]==result_df["answer_binary"]).sum()/len(result_df))